# Sentiment Analysis with BERT

- Framework: Pytorch and [Tez](https://github.com/abhishekkrthakur/tez)

In [1]:
%cd ..

/Users/tracesmith/Desktop/Trace/Coding/datascience-notebooks


In [2]:
%load_ext autoreload

In [21]:
import os
import torch
import pandas as pd
from nltk.corpus import movie_reviews 
from nlp.scripts.bert import BERTDataset,BERTClassifier,train_model

### Load Movie Reviews

In [5]:
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

In [6]:
df = pd.DataFrame(list(documents), columns=["Review","Label"])
df['Review'] = df['Review'].apply(lambda x: ' '.join(x))
df['Label'] = pd.Categorical(df['Label']).codes.astype(float)

### Split Train/Validation/Test

In [7]:
# train/validation split ratio
train_idx = int(df.shape[0] * 0.80)
val_idx = int(train_idx + (df.shape[0] - train_idx) * 0.50)

# Train 
X_train, y_train = df.iloc[:train_idx,0].values, df.iloc[:train_idx,1].values

# Validation
X_val, y_val = df.iloc[train_idx:val_idx,0].values, df.iloc[train_idx:val_idx,1].values

# Test
X_test, y_test = df.iloc[val_idx:,0].values, df.iloc[val_idx:,1].values

print(f'Train: {X_train.shape[0]} - Validation: {X_val.shape[0]} - Test: {X_test.shape[0]}')

Train: 1600 - Validation: 200 - Test: 200


### Train Model

In [ ]:
%autoreload
history = train_model(X_train,y_train,X_val,y_val,epochs=2)

### Evaluate Model

In [36]:
model = torch.load(os.path.join("model.bin"))
test_dataset = BERTDataset(X_test, y_test)
preds = model.predict(test_dataset, batch_size=32, n_jobs=-1, device="cpu")